In [2]:
import os
import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm

In [2]:
# Папки с изображениями и масками
image_dir = 'data/train/images'
mask_dir = 'data/train/masks'

In [3]:
# Списки имен файлов изображений и масок
image_filenames = [
    '9_2.tif',
    '6_2.tif',
    '6_1.tif',
    '5.tif',
    '4.tif',
    '2.tif',
    '1.tif'
]

In [4]:
# Поскольку имена масок должны совпадать с именами изображений, мы используем те же имена
mask_filenames = image_filenames.copy()

In [5]:

def create_combined_dataset(image_dir, mask_dir, image_filenames):
    """Создает комбинированный датасет из изображений и масок."""
    for filename in tqdm(image_filenames, desc="Processing files"):
        image_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename)  # Маска должна иметь то же имя, что и изображение

        # Открываем изображение
        with rasterio.open(image_path) as img:
            image_data = img.read()  # Чтение всех каналов изображения
            image_data = image_data.reshape(image_data.shape[0], -1).T  # Преобразуем в одномерный массив

        # Открываем маску
        with rasterio.open(mask_path) as mask:
            mask_data = mask.read(1)  # Чтение маски (один канал, бинарный)
            mask_data = mask_data.flatten()  # Приведение маски к одномерному виду

        # Объединяем данные изображения и маски
        combined_data = np.column_stack((image_data, mask_data))

        # Сохраняем в CSV
        combined_filename = f'combined_data_{filename[:-4]}.csv'  # Название файла без .tif
        save_to_csv(combined_data, combined_filename)

In [6]:
def save_to_csv(data, filename):
    """Сохраняет данные в CSV файл."""
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [7]:
create_combined_dataset(image_dir, mask_dir, image_filenames)

Processing files: 100%|██████████| 7/7 [13:49<00:00, 118.48s/it]


In [3]:
df = pd.read_csv('C:\\PFO\\combined_data_9_2.csv')
df

,0,1,2,3,4,5,6,7,8,9,10
0,330,426,316,431,378,462,176,476,355,275,1
1,385,473,440,431,378,462,532,476,355,275,0
2,386,490,512,578,636,729,810,807,532,408,0
3,419,480,524,578,636,729,818,807,532,408,0
4,378,434,444,489,532,547,678,639,480,380,0
...,...,...,...,...,...,...,...,...,...,...,...
262139,614,855,885,792,306,301,249,155,46,51,1
262140,583,817,852,767,330,321,264,194,63,51,1
262141,570,800,840,767,330,321,277,194,63,51,1
262142,592,830,877,780,310,305,262,169,49,57,1
